# First Lab: A Frontier LLM Project

## 1. Concepts

### 1.1. Types of prompts

Models like GPT have been trained to receive instructions in a particular way. They expect to receive:

- A **system** prompt that tells them what task they are performing and what tone they should use

- A **user** prompt that is the conversation starter that they should reply to

### 1.2. Messages

The API from OpenAI expects to receive messages in a particular structure. Many of the other APIs share this structure:

```python
[
    {"role": "system", "content": "system message goes here"},
    {"role": "user", "content": "user message goes here"}
]
```

## 2. Use Gemini API

In [ ]:
# 1. Check if the environment is set up correctly:

import os
from dotenv import load_dotenv
from scraper import fetch_website_contents
from IPython.display import Markdown, display
from openai import OpenAI

# This should NOT throw an error

In [ ]:
# 2. Load environment variables in a file called .env

load_dotenv(override=True)
api_key = os.getenv("GOOGLE_API_KEY")
base_url = os.getenv("GEMINI_BASE_URL")
model = os.getenv("GEMINI_MODEL", "gemini-3-flash-preview")

missing = []
if not api_key:
    missing.append("GOOGLE_API_KEY")
if not base_url:
    missing.append("GEMINI_BASE_URL")

if missing:
    raise ValueError(f"Missing required env vars: {', '.join(missing)}")

if api_key.strip() != api_key:
    raise ValueError("API key has leading/trailing whitespace; please clean it.")

print(f"API key found, using model '{model}' against {base_url}")

client = OpenAI(base_url=base_url, api_key=api_key)

In [ ]:
# 3. Create a message and a dictionary

message = "Hello, Gemini! This is my first ever message to you!"
messages = [{"role": "user", "content": message}]

messages

In [ ]:
# 4. Call the API

response = client.chat.completions.create(model=model, messages=messages)
response.choices[0].message.content

## 3. Use the Scrapper utility

> NOTE: this will only work on websites that can be scraped using this simplistic approach. Websites that are rendered with Javascript, like React apps, won't show up. There is a workaround that it will be in another file soon.

In [ ]:
# Define our system prompt

system_prompt = """
You are a snarky assistant that analyzes the contents of a website,
and provides a short, snarky, humorous summary, ignoring text that might be navigation related.
Respond in markdown. Do not wrap the markdown in a code block - respond just with the markdown.
"""

# Define our user prompt

user_prompt_prefix = """
Here are the contents of a website.
Provide a short summary of this website.
If it includes news or announcements, then summarize these too.
"""

In [ ]:
# Let's build useful messages for the API using a function and the Scrapper utility

def messages_for(website):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_prefix + website}
    ]

In [ ]:
cnn = fetch_website_contents("https://cnn.com/")
print(f"Fetched {len(cnn)} characters from CNN")
print("Preview (first 500 chars):")
print(cnn[:500] + "..." if len(cnn) > 500 else cnn)

messages_for(cnn)

In [ ]:
# Time to bring it all together

def summarize(url):
    website = fetch_website_contents(url)
    response = client.chat.completions.create(
        model=model,
        messages=messages_for(website)
    )
    return response.choices[0].message.content

In [ ]:
summarize("https://cnn.com/")

In [ ]:
# A function to display this nicely in the output, using markdown

def display_summary(url):
    """Fetch + summarize a URL and render the result as Markdown output."""
    try:
        summary = summarize(url)
        display(Markdown(summary))
    except Exception as exc:
        display(Markdown(f"**Error:** {exc}"))

In [ ]:
display_summary("https://manuel-valles.com/");